<H1><font color='green'>Working on EDA for Loan prediction data</font></H1>

In [1]:
# Mount your google drive to have access to data

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [9]:
#df = pd.read_csv('drive/My Drive/trainLoanPrediction.csv')
#print(df.head())
print(df.shape)

(614, 13)


In [4]:
# A quick look at all fields and unique values they contain
print(df.groupby('Gender')['Gender'].count())
print(df.groupby('Married')['Married'].count())
print(df.groupby('Dependents')['Dependents'].count())
print(df.groupby('Education')['Education'].count())
print(df.groupby('Self_Employed')['Self_Employed'].count())
print(df.groupby('Loan_Amount_Term')['Loan_Amount_Term'].count())
print(df.groupby('Credit_History')['Credit_History'].count())
print(df.groupby('Property_Area')['Property_Area'].count())

Gender
Female    112
Male      489
Name: Gender, dtype: int64
Married
No     213
Yes    398
Name: Married, dtype: int64
Dependents
0     345
1     102
2     101
3+     51
Name: Dependents, dtype: int64
Education
Graduate        480
Not Graduate    134
Name: Education, dtype: int64
Self_Employed
No     500
Yes     82
Name: Self_Employed, dtype: int64
Loan_Amount_Term
12.0       1
36.0       2
60.0       2
84.0       4
120.0      3
180.0     44
240.0      4
300.0     13
360.0    512
480.0     15
Name: Loan_Amount_Term, dtype: int64
Credit_History
0.0     89
1.0    475
Name: Credit_History, dtype: int64
Property_Area
Rural        179
Semiurban    233
Urban        202
Name: Property_Area, dtype: int64


In [7]:
# A look at the null values in columns.
print('Total values: %d, total nulls: %d' %(len(df['Gender']),  df['Gender'].isna().sum()))
print('Total values: %d, total nulls: %d' %(len(df['Married']),  df['Married'].isna().sum()))
print('Total values: %d, total nulls: %d' %(len(df['Dependents']),  df['Dependents'].isna().sum()))
print('Total values: %d, total nulls: %d' %(len(df['Education']),  df['Education'].isna().sum()))
print('Total values: %d, total nulls: %d' %(len(df['Self_Employed']),  df['Self_Employed'].isna().sum()))
print('Total values: %d, total nulls: %d' %(len(df['Loan_Amount_Term']),  df['Loan_Amount_Term'].isna().sum()))
print('Total values: %d, total nulls: %d' %(len(df['Credit_History']),  df['Credit_History'].isna().sum()))
print('Total values: %d, total nulls: %d' %(len(df['Property_Area']),  df['Property_Area'].isna().sum()))

# There aren't many null values, we can assume them to be another class for the sake of baselining

Total values: 614, total nulls: 13
Total values: 614, total nulls: 3
Total values: 614, total nulls: 15
Total values: 614, total nulls: 0
Total values: 614, total nulls: 32
Total values: 614, total nulls: 14
Total values: 614, total nulls: 50
Total values: 614, total nulls: 0


In [12]:
X = pd.get_dummies(df, dummy_na = True, drop_first=True, columns = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Loan_Amount_Term', 'Credit_History', 'Property_Area'])
print(X.head())
print(X.shape)

    Loan_ID  ApplicantIncome  CoapplicantIncome  LoanAmount Loan_Status  \
0  LP001002             5849                0.0         NaN           Y   
1  LP001003             4583             1508.0       128.0           N   
2  LP001005             3000                0.0        66.0           Y   
3  LP001006             2583             2358.0       120.0           Y   
4  LP001008             6000                0.0       141.0           Y   

   Gender_Male  Gender_nan  Married_Yes  Married_nan  Dependents_1  \
0            1           0            0            0             0   
1            1           0            1            0             1   
2            1           0            1            0             0   
3            1           0            1            0             0   
4            1           0            0            0             0   

         ...          Loan_Amount_Term_240.0  Loan_Amount_Term_300.0  \
0        ...                               0            

In [0]:
# Baselining
X['Loan_Status'] = X.Loan_Status.map(dict(Y=1, N=0))
Y = X['Loan_Status']
X.drop(columns=['Loan_Status', 'Loan_ID'], inplace=True)
classifier = LogisticRegression(random_state=11)
trainX, testX, trainY, testY = train_test_split(X, Y, random_state=11)

In [19]:
classifier.fit(trainX, trainY)

ValueError: ignored